In [20]:
# https://www.kaggle.com/charlotteloh/cord-19-tf-idf-bert-q-a#Detailed-Approach:
import numpy as np
import pandas as pd
import glob
import json
import math
import os
root_dir = 'E:/2000-2010 recurrent glioblastoma & drug target (WoS)/json_WoS'
doc_paths = glob.glob(f'{root_dir}/*.json')

def get_text(doc_paths):
# Genertaing metafile form EndnoteXML
    from xml.etree.ElementTree import parse
    document = parse('E:/2000-2010 recurrent glioblastoma & drug target (WoS)/endnoteXML/rGBM-Web of Science (2000-2021 recurrent glioblastoma)_fulltext.xml')
    root =  document.getroot()

    url = []
    title = []
    abstract = []
    pdf = []
    for object in root.iter("record"):
        abstract.append(object.find("abstract").findtext("style"))
        title.append(object.find("titles").find("title").findtext("style"))
        url.append(object.find("urls").find("related-urls").findall("url")[1].findtext("style"))
        pdf.append(os.path.basename(object.find("urls").find("pdf-urls").findtext("url")))

    metafile  = pd.DataFrame({'title': title, 'abstract':abstract, 'doi':url, 'pdf':pdf})
# Initialise the full_text array of articles
    full_text=[]
    for jsonfile in doc_paths:
        with open(jsonfile) as f:
            file = json.load(f)
            # article_series = sub_df[sub_df['sha'].str.contains(sha, na=False)]
            print(file['paper_id'])
            matched = (file['paper_id'].lower() + '.pdf') == metafile['pdf'].str.lower()
            index = np.flatnonzero(matched)[0]
            title = metafile['title'][index]
            abstract = metafile['abstract'][index]
            url =  metafile['doi'][index]

            for text_part in file['pdf_parse']['body_text']:
                text = text_part['text']
                 # remove citations from each paragraph
                for citation in text_part['cite_spans']:
                    text = text.replace(citation['text'], "")
                full_text.append([title, text, abstract, url])

    return full_text

full_text = get_text(doc_paths)
full_text_df = pd.DataFrame(full_text, columns=['title','text', 'abstract', 'url'])


full_text_df = full_text_df.replace(["\r", ":", r"Purpose", r'Background', r"Objective",r"Objectives","Simple Summary",r"Abstract", r"Keyword",r"Method", r"Methods", r"Results",r"Conclusions",r"Result",r"Conclusion"],
                                     value ='', regex=True)


pd.set_option('display.max_colwidth', 50)
full_text_df.head()
print("No. of paragraphs: ",len(full_text_df))



Agarwal-2011-Delivery of molecularly targeted
Sestito-2018-New Multitarget Approaches in the
Kinfe-2020-The Diagnostic and Therapeutic Role
Chang-2004-Phase I_pharmacokinetic study of CC
Ho-2015-Toxicity evaluation of prolonged conve
Nghiemphu-2018-Phase I study of sorafenib and
Mendiburu-Elica-2012-Systemic rapamycin alone
Parker-2013-The tumorigenic FGFR3-TACC3 gene f
Staedtke-2016-Investigational new drugs for br
Sharma-2018-Receptor-Targeted Glial Brain Tumo
Zhang-2010-XL-184, a MET, VEGFR-2 and RET kina
Hu-2007-Calcium-activated potassium channels m
Reardon-2017-Efficacy and safety results of AB
Alamir-2018-In situ characterization of stem c
Hays-2017-Aptamers and Glioblastoma_ Their Pot
Moustakas-2010-New treatment options in the ma
Barua-2016-A novel implantable catheter system
Skaga-2019-The efficacy of a coordinated pharm
Burghardt-2021-Endoglin and TGF-beta signaling
Martinho-2013-In Vitro and In Vivo Analysis of
D'Amico-2020-Super selective intra-arterial ce
Takahashi-2014-R

In [1]:
import requests
import spacy
nlp = spacy.load("en_core_web_sm")

abstract = full_text_df.drop_duplicates(['abstract'])

index_drug = []
drug = []
pos_drug = []
block_drug = []

done_list = []
n = 1

NameError: name 'full_text_df' is not defined

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, AutoTokenizer
from transformers import DataCollatorForTokenClassification, pipeline
tokenizer = AutoTokenizer.from_pretrained("E:/2021 NLP/Trained_Chem/", max_length=256)
model = AutoModelForTokenClassification.from_pretrained("E:/2021 NLP/Trained_Chem/")
ner = pipeline("ner", model=model, tokenizer=tokenizer)

def query_raw(text):
    return ner(text)

import pdb
for j in abstract.index:
    sentences = nlp(abstract['abstract'][j])
    assert sentences.has_annotation("SENT_START")
    my_list = list(sentences.sents)

    for div in my_list:
        dict = query_raw(str(div))
        print(str(div))

        if len(dict) == 0:
            continue

        for i in range(0,len(dict)):
            if dict[i]['entity'] =='Label_1' or ((dict[i]['entity'] =='Label_2') and dict[i]['word'].startswith("#")):
                pre_word = drug.pop()
                drug.append(pre_word + dict[i]['word'].replace("##",""))

            elif ((dict[i]['entity'] =='Label_2') and dict[i]['word'].startswith("-")):
                pre_word = drug.pop()
                drug.append(pre_word + dict[i]['word'])

            elif ((dict[i]['entity'] =='Label_2') and dict[i]['word'].isdigit()):
                pre_word = drug.pop()
                drug.append(pre_word + '/' + dict[i]['word'])

            elif dict[i]['entity'] =='Label_2':
                drug.append(dict[i]['word'])
                pos_drug.append(dict[i]['start'])
                block_drug.append(str(div))
                index_drug.append(j)

absdrug = pd.DataFrame({'index': index_drug, 'word': drug, 'block':block_drug, 'pos':pos_drug})
absdrug = absdrug.set_index('index')
absdrug.to_csv('E:/2021 NLP/absdrug_Sap.csv', header=True)

In [ ]:
index_gene = []
genes = []
pos_gene = []
block_gene = []

from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, AutoTokenizer
from transformers import DataCollatorForTokenClassification, pipeline
tokenizer = AutoTokenizer.from_pretrained("E:/2021 NLP/Trained_Gene/", max_length=256)
model = AutoModelForTokenClassification.from_pretrained("E:/2021 NLP/Trained_Gene/")
ner = pipeline("ner", model=model, tokenizer=tokenizer)

def query_raw(text):
    return ner(text)

for j in abstract.index:
    sentences = nlp(abstract['abstract'][j])
    assert sentences.has_annotation("SENT_START")
    my_list = list(sentences.sents)

    for div in my_list:
        dict = query_raw(str(div))
        print(str(div))

        if len(dict) == 0:
            continue

        for i in range(0,len(dict)):
            if dict[i]['entity'] =='LABEL_1' or ((dict[i]['entity'] =='LABEL_2') and dict[i]['word'].startswith("#")):
                pre_word = genes.pop()
                genes.append(pre_word + dict[i]['word'].replace("##",""))

            elif (dict[i]['entity'] =='LABEL_2') and dict[i]['word'].startswith("-"):
                pre_word = genes.pop()
                genes.append(pre_word + dict[i]['word'])

            elif (dict[i]['entity'] =='LABEL_2') and dict[i]['word'].isdigit():
                print(i)
                pre_word = genes.pop()
                genes.append(pre_word + '/' + dict[i]['word'])

            elif dict[i]['entity'] =='LABEL_2':
                genes.append(dict[i]['word'])
                pos_gene.append(dict[i]['start'])
                block_gene.append(str(div))
                index_gene.append(j)

absgene = pd.DataFrame({'index': index_gene, 'word': genes, 'block':block_gene, 'pos':pos_gene})
absgene = absgene.set_index('index')
absgene.to_csv('E:/2021 NLP/absgene_Sap.csv', header=True)


In [ ]:
import pandas as pd
absgene = pd.read_csv('E:/2021 NLP/absgene_Sap.csv')
absdrug = pd.read_csv('E:/2021 NLP/absdrug_Sap.csv')

dat_gene = absgene.set_index('index')
dat_drug = absdrug.set_index('index')

In [ ]:
# This function makes the 2D quilt plot for showing co-occurences at block
# or sentence level of various classes of search terms
def Make2DPlot(dat_joined, factor1, factor2, single_sentence_plots=False):
    import pylab
    import numpy as pd

    if(single_sentence_plots):
        grouped = dat_joined[dat_joined.same_sentence==True].groupby([factor1, factor2])
    else:
        grouped = dat_joined.groupby([factor1,factor2])

    Values = grouped.count().values[:,0]

    Index=grouped.count().index
    Index1=[]
    Index2=[]
    for i in Index:
        Index1.append(i[0])
        Index2.append(i[1])

    Uniq1=np.unique(Index1)
    Uniq2=np.unique(Index2)

    for i in range(0,len(Index1)):
        Index1[i]=np.where(Index1[i]==Uniq1)[0][0]
        Index2[i]=np.where(Index2[i]==Uniq2)[0][0]


    pylab.figure(figsize=(15,15),dpi=200)
    hist=pylab.hist2d(Index1,Index2, (range(0,len(Uniq1)+1),range(0,len(Uniq2)+1)), weights=Values,cmap='Blues')
    pylab.xticks(np.arange(0,len(Uniq1))+0.5, Uniq1,rotation=90)
    pylab.yticks(np.arange(0,len(Uniq2))+0.5, Uniq2)
    pylab.clim(0,np.max(hist[0])*1.5)
    print("done!!")
    for i in range(0,len(Uniq1)):
        for j in range(0,len(Uniq2)):
            pylab.text(i+0.5,j+0.5,int(hist[0][i][j]),ha='center',va='center')
    print("done!2!")
    pylab.colorbar()
    if(single_sentence_plots):
        pylab.title(factor1+" and " +factor2+" in One Sentence")
        pylab.tight_layout()
        pylab.savefig("Overlap"+factor1+"_Vs_"+factor2+"_2D_sentence.png",bbox_inches='tight',dpi=200)
    else:
        pylab.title(factor1+" and " +factor2+" in One Block")
        pylab.tight_layout()
        pylab.savefig("Overlap"+factor1+"_Vs_"+factor2+"_2D_block.png",bbox_inches='tight',dpi=200)


def SameSentenceCheck(block1, block2):
    if block1 == block2:
        return 1
    return 0

dat_gene.word.value_counts()
dat_drug.word.value_counts()

dat_joined_vt=dat_gene.join(dat_drug, rsuffix='_drug',lsuffix="_gene")
dat_joined_vt=dat_joined_vt[dat_joined_vt.notna().word_gene  & dat_joined_vt.notna().word_drug]

InOneSentence=[]
for idx, member in dat_joined_vt.iterrows():
    InOneSentence.append(SameSentenceCheck(member.block_drug, member.block_gene))

dat_joined_vt.insert(len(dat_joined_vt.columns),'same_sentence',InOneSentence)
dat_joined_vt.to_csv("Overlaps_Drug_Gene.csv")


In [ ]:
dat_joined_vt = pd.read_csv('E:/2021 NLP/Overlaps_Drug_Gene.csv')
#dat_joined_vt['count'] = dat_joined_vt.groupby(['word_gene','word_drug']).same_sentence.transform(np.count_nonzero)
grouped = dat_joined_vt[dat_joined_vt.same_sentence==1]
grouped.reset_index()
grouped = grouped.groupby(['word_gene','word_drug']).agg({'index': lambda x: set(tuple(x))}).reset_index()
grouped['count'] = [len(c) for c in grouped['index']]

In [ ]:
selecte_index = set(grouped.iloc[475]['index'])
abstract['abstract'][selecte_index]


In [ ]:
# Make2DPlot(dat_joined,"word_gene","word_drug",single_sentence_plots=True)
